In [ ]:
import tensorflow
import pygad.kerasga
import numpy
import pygad
import os
from sklearn.model_selection import train_test_split
import pathlib

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

# Load Data

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tensorflow.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
data_dir

In [ ]:
len(list(data_dir.glob('*/*.jpg')))

In [ ]:
def dir_train_test_split(split, directory):
    file_paths = []
    for fileName in os.listdir(directory):
        file_paths.append(os.path.join(directory, fileName))
    data = numpy.array(file_paths)
    return train_test_split(data,test_size=split) 

In [ ]:
def load_images(file_paths):
    data = []
    for filePath in list(file_paths):
        img = tensorflow.keras.preprocessing.image.load_img(filePath)
        img_array = tensorflow.keras.preprocessing.image.img_to_array(img)
        normalized_img = tensorflow.image.resize_with_crop_or_pad(img_array, img_height, img_width)
        data.append(normalized_img)
    return numpy.array(data)

In [ ]:
def load_labels(file_paths):
    labels = []
    for path in file_paths:
        if "daisy" in path:
            labels.append(0)
        elif "dandelion" in path:
            labels.append(1)
        elif "roses" in path:
            labels.append(2)
        elif "sunflowers" in path:
            labels.append(3)
        elif "tulips" in path:
            labels.append(4)
        else:
            labels.append(6)
    return tensorflow.keras.utils.to_categorical(numpy.array(labels))

In [ ]:
train = []
test = []
for label in ["daisy", "dandelion", "roses", "sunflowers", "tulips"]:
    current_path = pathlib.Path.joinpath(data_dir, label)
    current_test, current_train = dir_train_test_split(0.8, current_path)
    train = numpy.concatenate((train, current_train))
    test = numpy.concatenate((test, current_test))

    
train_data = load_images(train)
test_data = load_images(test)
train_labels = load_labels(train)
test_labels = load_labels(test)


In [ ]:
train_data.shape

In [ ]:
train_labels.shape

# Tensorflow Model

In [ ]:
def setup_model(class_names):
    num_classes = len(class_names)

    model = tensorflow.keras.models.Sequential([
      tensorflow.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      tensorflow.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(),
      tensorflow.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(),
      tensorflow.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
      tensorflow.keras.layers.MaxPooling2D(),
      tensorflow.keras.layers.Flatten(),
      tensorflow.keras.layers.Dense(128, activation='relu'),
      tensorflow.keras.layers.Dense(num_classes)
    ])
    
    return model

# Simple Model Training

In [ ]:
simple_model = setup_model(class_names)
simple_model.compile(optimizer='adam',
    loss=tensorflow.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

simple_model.fit(
    train_data,
    train_labels,
    validation_split=0.1,
    epochs=10
)

In [ ]:
_, baseline_model_accuracy = simple_model.evaluate(test_data, test_labels, verbose=0)
baseline_model_accuracy

# Genetic Algorithm Optimized Classification

In [ ]:
def fitness_func(solution, sol_idx):
    global train_data, train_labels, keras_ga, model

    predictions = pygad.kerasga.predict(model=model, solution=solution, data=train_data)

    ce = tensorflow.keras.losses.CategoricalCrossentropy()
    solution_fitness = 1.0 / (ce(train_labels, predictions).numpy() + 0.00000001)

    return solution_fitness

In [ ]:
def callback_generation(ga_instance):
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))

In [ ]:
ga_model = setup_model(class_names)

keras_ga = pygad.kerasga.KerasGA(model=ga_model,num_solutions=10)

num_generations = 100 
num_parents_mating = 5 
initial_population = keras_ga.population_weights 

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation)

ga_instance.run()

In [ ]:
ga_instance.plot_fitness(title="PyGAD & Keras - Iteration vs. Fitness", linewidth=4)

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

In [ ]:
predictions = pygad.kerasga.predict(model=model,solution=solution,data=test_data)

cce = tensorflow.keras.losses.CategoricalCrossentropy()
print("Categorical Crossentropy : ", cce(test_labels, predictions).numpy())

ca = tensorflow.keras.metrics.CategoricalAccuracy()
ca.update_state(test_labels, predictions)
accuracy = ca.result().numpy()
print("Accuracy : ", accuracy)